In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

In [2]:
#Import Data
df=pd.read_csv('Omkar01212020.csv')

In [3]:
#PreProcessing Data 
##Removing nan and replacing with 0 for initial sum OR -1 in case of PrevClaimMonth OR -2 in case of PrePreClaimMonth
##And converting Time stamps to Ordinal values
df['Prev1Claim'].fillna(0,inplace=True)
df['Prev2Claim'].fillna(0,inplace=True)
df['RollingSum'].fillna(0,inplace=True)

df['PreClaimMonthDiff'].fillna(-1,inplace=True)
df['PrePreClaimMonthDiff'].fillna(-2,inplace=True)

df['EnrollDate'] = pd.to_datetime(df['EnrollDate'])
df['EnrollDate']=df['EnrollDate'].map(dt.datetime.toordinal)

df['ClaimDate'] = pd.to_datetime(df['ClaimDate'])
df['ClaimDate']=df['ClaimDate'].map(dt.datetime.toordinal)



In [4]:
#Spitting Train and Test data
#Train data includes all rows that were provided in Pet and Claim Data set after doign a join deriving new columns
dfTrain=df.query('ClaimDate!=737241')
dfTest=df.query('ClaimDate==737241')

In [5]:
#DEfining indipendent and dependent variables for Training Data set
X = dfTrain[['EnrollDate', 'Species', 'Breed', 'AgeAtEnroll', 'Prev1Claim',
       'Prev2Claim', 'MonthAfterEnroll','RollingSum','DiffInClaim',
       'ClaimDate', 'PreClaimMonthDiff', 'PrePreClaimMonthDiff']].values
y = dfTrain[['ClaimPerMonth']].values

In [6]:
#DEfining indipendent variables for Test Data set
Xtest = dfTest[['EnrollDate', 'Species', 'Breed', 'AgeAtEnroll', 'Prev1Claim',
       'Prev2Claim', 'MonthAfterEnroll','RollingSum','DiffInClaim',
       'ClaimDate', 'PreClaimMonthDiff', 'PrePreClaimMonthDiff']].values

In [7]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoderS = LabelEncoder()
X[:,1] = labelencoderS.fit_transform(X[:,1])
labelencoderB = LabelEncoder()
X[:,2] = labelencoderB.fit_transform(X[:,2])
labelencoderA = LabelEncoder()
X[:,3] = labelencoderA.fit_transform(X[:,3])

In [8]:
#Encoding Test Data Set for Categorical Data
Xtest[:,1] = labelencoderS.transform(Xtest[:,1])
Xtest[:,2] = labelencoderB.transform(Xtest[:,2])
Xtest[:,3] = labelencoderA.transform(Xtest[:,3])

In [9]:
# Fitting Random Forest Regression to the Train dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000,n_jobs=-1, random_state = 0)
regressor.fit(X, y)

C:\Users\opawar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [10]:
# Predicting the Claim Amount for 2019-07-01
y_pred = regressor.predict(Xtest)

In [11]:
#Peplacing ClaimPerMonth in test Dataset with Predicted ClaimPerMonth
preDF=dfTest
preDF['ClaimPerMonth']=y_pred

C:\Users\opawar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
#Creating Answer Dataframe for ClaimPerMonth for 2019-07-01
Answer=preDF[['PetID','ClaimPerMonth','ClaimDate']]
Answer['ClaimDate']=Answer['ClaimDate'].map(dt.datetime.fromordinal)

C:\Users\opawar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
Answer

,PetID,ClaimPerMonth,ClaimDate
25,1009,403.972930,2019-07-01
33,1010,1039.984090,2019-07-01
41,1011,2350.370100,2019-07-01
47,1017,1097.946860,2019-07-01
50,1022,2345.027800,2019-07-01
...,...,...,...
25285,9987,1513.839880,2019-07-01
25294,9991,1031.464190,2019-07-01
25297,9992,2187.589280,2019-07-01
25300,9995,346.198047,2019-07-01


In [15]:
Answer.to_csv('Answer.csv',index=False)